In [1]:
import pyspark.sql.functions as F
import json
from cdk.services.api.energi_data_service import EnergiDataService
from cdk.common_modules.access.secrets import Secrets
from cdk.common_modules.spark.spark_config import SparkConfig
from cdk.common_modules.spark.spark_session_builder import SparkSessionBuilder

storage_account_name = "adlsthuehomelakehousedev"

# Set Spark configurations
spark_config = SparkConfig(Secrets())

# Add jars to install
spark_config.add_jars_to_install(['hudi', 'azure_storage', 'delta'])

# Add storage account access
spark_config.add_storage_account_access(storage_account_name, method='access_key')

# Build SparkSession
spark = SparkSessionBuilder("ViewAzureData", spark_config).build()


dataset_name = "delta_state"
dataset_path = f"delta" #/20231210/{dataset_name}"
container_name = "utility"

utility_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{dataset_path}/{dataset_name}"




INFO:root:Loading secret ADLS_adlsthuehomelakehousedev_access_key


:: loading settings :: url = jar:file:/opt/bitnami/python/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
org.apache.hadoop#hadoop-azure added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7290faf0-9c10-4a2a-bd7c-88b921c3f4ec;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
	found org.apache.hadoop#hadoop-azure;3.3.3 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.13.2 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;

23/12/12 20:01:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
delta_state = spark.read.format("hudi").load(utility_path)

delta_state.show()

23/12/12 20:02:56 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
23/12/12 20:02:56 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file


+-------------------+--------------------+--------------------+----------------------+--------------------+------------------+------------------+-------+----------------+
|_hoodie_commit_time|_hoodie_commit_seqno|  _hoodie_record_key|_hoodie_partition_path|   _hoodie_file_name|   FromDatasetName|     ToDatasetName|ToLayer|      DeltaState|
+-------------------+--------------------+--------------------+----------------------+--------------------+------------------+------------------+-------+----------------+
|  20231210211656391|20231210211656391...|FromDatasetName:s...|                      |d9a789bd-e614-420...|stock_transactions|stock_transactions| bronze|2023-12-10T00:00|
+-------------------+--------------------+--------------------+----------------------+--------------------+------------------+------------------+-------+----------------+

